In [16]:
from osgeo import gdal
from matplotlib import pyplot as plt
from skimage.filters.rank import entropy
from skimage.morphology import disk
import cv2
import numpy as np
from skimage.util import img_as_ubyte
    
%matplotlib qt5

# Open Data

LANDSAT 7 images from 2008 to 2011 

In [8]:
year = gdal.Open('KNP2009.tif')

band1 = year.GetRasterBand(1)
band2 = year.GetRasterBand(2)
band3 = year.GetRasterBand(3)
band4 = year.GetRasterBand(4)
band5 = year.GetRasterBand(5)
band6 = year.GetRasterBand(6)

band1_array = img_as_ubyte(band1.ReadAsArray())
band2_array = img_as_ubyte(band2.ReadAsArray())
band3_array = img_as_ubyte(band3.ReadAsArray())
band4_array = img_as_ubyte(band4.ReadAsArray())
band5_array = img_as_ubyte(band5.ReadAsArray())
band6_array = img_as_ubyte(band6.ReadAsArray())

LANDSAT 8 images from 2013 to 2018

In [624]:
year = gdal.Open('KNP2018.tif')

band1 = year.GetRasterBand(6)
band2 = year.GetRasterBand(5)
band3 = year.GetRasterBand(4)
band4 = year.GetRasterBand(3)
band5 = year.GetRasterBand(2)
band6 = year.GetRasterBand(1)

band1_array = img_as_ubyte(band1.ReadAsArray())
band2_array = img_as_ubyte(band2.ReadAsArray())
band3_array = img_as_ubyte(band3.ReadAsArray())
band4_array = img_as_ubyte(band4.ReadAsArray())
band5_array = img_as_ubyte(band5.ReadAsArray())
band6_array = img_as_ubyte(band6.ReadAsArray())

# River Extraction

In [17]:
from numpy import save,load
from skimage.morphology import area_opening,area_closing,remove_small_holes,erosion,dilation,disk,convex_hull_object,square

In [18]:
image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band6_array
image[:,:,1] = band5_array
image[:,:,2] = band4_array

In [19]:
lo = (band2_array/255) + 2.5*(band3_array/255)-1.5*(band5_array/255 + band6_array/255)-0.25*(band6_array/255)

In [20]:
b_lo = (lo>0.170)*1

In [21]:
selem = disk(15)
water = area_opening(area_closing(b_lo))

In [22]:
#mask1 =convex_hull_object(water, connectivity=2)

In [23]:
plt.imshow(water, cmap = plt.cm.gray)
plt.show()

In [24]:
edge_water = cv2.Canny((water*255).astype(np.uint8), 50, 150)

In [25]:
contours,hierarchy = cv2.findContours(edge_water,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [26]:
k = cv2.drawContours(image.astype(int), contours, -1, (255, 0, 0), 1)

In [27]:
plt.imshow(k.astype(int), cmap = plt.cm.gray)
plt.show()

In [28]:
save('river_mask_2018',water)

In [4]:
a =load('river_mask_2018.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'river_mask_2018.npy'

In [339]:
plt.imshow(a, cmap = 'gray')
plt.show()

# Sand Bed Extraction

In [3]:
from skimage.exposure import adjust_gamma, adjust_sigmoid, adjust_log
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

In [625]:
image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band1_array.astype('int')
image[:,:,1] = band3_array.astype('int')
image[:,:,2] = band6_array.astype('int')

In [626]:
# Contrast Enhancement using log correction
gray = rgb2gray(image.astype(int))
enh_img_log = (100000000*adjust_log(gray,100)).astype('int')

In [627]:
th = threshold_otsu(enh_img_log)
print(th)

251


In [628]:
b_sand = dilation(erosion((enh_img_log>th)*1, disk(1)),disk(1))

In [629]:
plt.imshow(b_sand, cmap = 'gray')
plt.show()

In [630]:
plt.hist(enh_img_log.ravel(), bins = 256)
plt.title('Kaziranga Image 2008')
plt.show()

In [631]:
edge_sand = cv2.Canny((b_sand*255).astype(np.uint8), 50, 150)

In [632]:
contours,hierarchy = cv2.findContours(edge_sand,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [633]:
k_s = cv2.drawContours(image.astype(int), contours, -1, (255, 0, 0), 1)

In [634]:
plt.imshow(k_s.astype(int), cmap = plt.cm.gray)
plt.show()

In [635]:
save('sand_mask_2018',b_sand)

In [623]:
a =load('sand_mask_2017.npy')
plt.imshow(a, cmap = 'gray')
plt.show()

# Combining Two Masks

In [5]:
import os
from skimage.morphology import remove_small_objects, remove_small_holes

In [6]:
def load_images_from_folder(folder1):
    images = []
    for filename1 in os.listdir(folder1):
        img = load(os.path.join(folder1,filename1))
        if img is not None:
            images.append(img)
    return images

path_sand="D:\\Kaziranga\\sand"
sand= load_images_from_folder(path_sand)
path_river="D:\\Kaziranga\\river"
river= load_images_from_folder(path_river)

In [7]:
def combineMask(river_mask,sand_mask):
    images = []
    for i in range(0,10):
        coastal = river_mask[i]|sand_mask[i]
        images.append(1-coastal)
    return images

In [8]:
forest = combineMask(river,sand)

In [9]:
def visualizeForest(forest):
    plt.figure(1)
    plt.imshow(forest[0], cmap = 'gray')
    plt.figure(2)
    plt.imshow(forest[1], cmap = 'gray')
    plt.figure(3)
    plt.imshow(forest[2], cmap = 'gray')
    plt.figure(4)
    plt.imshow(forest[3], cmap = 'gray')
    plt.figure(5)
    plt.imshow(forest[4], cmap = 'gray')
    plt.figure(6)
    plt.imshow(forest[5], cmap = 'gray')
    plt.figure(7)
    plt.imshow(forest[6], cmap = 'gray')
    plt.figure(8)
    plt.imshow(forest[7], cmap = 'gray')
    plt.figure(9)
    plt.imshow(forest[8], cmap = 'gray')
    plt.figure(10)
    plt.imshow(forest[9], cmap = 'gray')
    
    plt.show()


In [10]:
visualizeForest(forest)

In [11]:
improved = []
c = 0
for i in forest:
       
    m_im = area_opening(i)
    mask =remove_small_holes(remove_small_objects(m_im,4000),4000)
    c = c+1
    print('Processed image---->', c)
    improved.append(mask)
    
    

C:\Users\Asus\anaconda3\envs\geo_py32\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  
C:\Users\Asus\anaconda3\envs\geo_py32\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  


Processed image----> 1
Processed image----> 2
Processed image----> 3
Processed image----> 4
Processed image----> 5
Processed image----> 6
Processed image----> 7
Processed image----> 8
Processed image----> 9
Processed image----> 10


In [12]:
visualizeForest(improved)

In [53]:
len(improved)

10

In [33]:
year = gdal.Open('KNP2018.tif')

band1 = year.GetRasterBand(6)
band2 = year.GetRasterBand(5)
band3 = year.GetRasterBand(4)
band4 = year.GetRasterBand(3)
band5 = year.GetRasterBand(2)
band6 = year.GetRasterBand(1)

band1_array = img_as_ubyte(band1.ReadAsArray())
band2_array = img_as_ubyte(band2.ReadAsArray())
band3_array = img_as_ubyte(band3.ReadAsArray())
band4_array = img_as_ubyte(band4.ReadAsArray())
band5_array = img_as_ubyte(band5.ReadAsArray())
band6_array = img_as_ubyte(band6.ReadAsArray())

image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band1_array.astype('int')
image[:,:,1] = band3_array.astype('int')
image[:,:,2] = band6_array.astype('int')


In [20]:
edge = cv2.Canny((improved[0]*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k1 = cv2.drawContours(image.astype(int), contours, -1, (255, 0, 0), 1)

plt.imshow(edge.astype(int), cmap = plt.cm.gray)
plt.show()

In [21]:
def Find_Kaziranga_Edge(edge_image):
    for col in range(edge_image.shape[1]-1,-1,-1):
        state = 1
        for row in range(edge_image.shape[0]-1,-1,-1):
            #print(edge_image[row][col])
            if edge_image[row][col] == 255:
                state = 0
            edge_image[row][col] = state
    return edge_image
        

In [22]:
edge_2008 = cv2.Canny((improved[0]*255).astype(np.uint8), 50, 150)
new_edge2008 = Find_Kaziranga_Edge(edge_2008*1)

edge_2018 = cv2.Canny((improved[9]*255).astype(np.uint8), 50, 150)
new_edge2018 = Find_Kaziranga_Edge(edge_2018*1)

In [29]:
plt.imshow(new_edge2018.astype('int'))
plt.show()

In [28]:
edge_2008 = cv2.Canny((new_edge2008*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(edge_2008,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k3 = cv2.drawContours(image.astype(int), contours, -1, (255, 0, 0), 1)

edge_2018 = cv2.Canny((new_edge2018*255).astype(np.uint8), 50, 150)
contours,hierarchy = cv2.findContours(edge_2018,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
k4 = cv2.drawContours(k3.astype(int), contours, -1, (255, 255, 0), 1)

plt.imshow(k4.astype(int), cmap = plt.cm.gray)
plt.show()

# Storing Mask of Land

Step 1 : Aquiring the mask and storing it in an array

In [36]:
from skimage.morphology import flood_fill

In [89]:
def findArray(image):
    
    img_boundary_steps = np.array([np.sum(image[:,col]) for col in range(0, image.shape[1])])
    return img_boundary_steps

In [90]:
edge_2018 = cv2.Canny((improved[9]*255).astype(np.uint8), 50, 150)
new_edge2018 = Find_Kaziranga_Edge(edge_2018*1)
array_boundary = findArray(new_edge2018)

In [91]:
plt.imshow(new_edge2018)
plt.show()